In [1]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter
import shutil
import os
import csv
from io import StringIO
import random
import re
from Bio.SeqRecord import SeqRecord
import torch
from pycaret.classification import *
import json
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("success_download.csv")
df

,ID,Name,Class,Family,Data type,Uniprot ID
0,MA0002.1,RUNX1,Runt domain factors,Runt-related factors,SELEX,Q01196
1,MA0003.2,TFAP2A,Basic helix-span-helix factors (bHSH),AP-2,ChIP-seq,P05549
2,MA0003.4,TFAP2A,Basic helix-span-helix factors (bHSH),AP-2,ChIP-seq,P05549
3,MA0007.2,AR,Nuclear receptors with C4 zinc fingers,Steroid hormone receptors (NR3),ChIP-seq,P10275
4,MA0014.2,PAX5,Paired box factors,Paired domain only,ChIP-seq,Q02548
...,...,...,...,...,...,...
325,MA1973.1,ZKSCAN3,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q9BRR0
326,MA1974.1,ZNF211,C2H2 zinc finger factors,Factors with multiple dispersed zinc fingers,NaN,Q13398
327,MA1979.1,ZNF416,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q8NA42
328,MA1983.1,ZNF582,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q96NG8


In [6]:
data = df[df['Class'] == 'C2H2 zinc finger factors']
data

,ID,Name,Class,Family,Data type,Uniprot ID
23,MA0039.3,KLF4,C2H2 zinc finger factors,Three-zinc finger Kruppel-related,ChIP-seq,O43474
24,MA0039.4,KLF4,C2H2 zinc finger factors,Three-zinc finger Kruppel-related,ChIP-seq,O43474
34,MA0056.1,MZF1,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,SELEX,P28698
35,MA0056.2,MZF1,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,P28698
51,MA0073.1,RREB1,C2H2 zinc finger factors,Factors with multiple dispersed zinc fingers,SELEX,Q92766
...,...,...,...,...,...,...
325,MA1973.1,ZKSCAN3,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q9BRR0
326,MA1974.1,ZNF211,C2H2 zinc finger factors,Factors with multiple dispersed zinc fingers,NaN,Q13398
327,MA1979.1,ZNF416,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q8NA42
328,MA1983.1,ZNF582,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,NaN,Q96NG8


In [20]:
train_df, test_df = train_test_split(data, test_size=9, random_state=10)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [21]:
train_df

,ID,Name,Class,Family,Data type,Uniprot ID
0,MA1597.1,ZNF528,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,Q3MIS6
1,MA1585.1,ZKSCAN1,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,P17029
2,MA1719.1,ZNF816,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,Q0VGE8
3,MA0162.2,EGR1,C2H2 zinc finger factors,Three-zinc finger Kruppel-related,ChIP-seq,P18146
4,MA0079.2,SP1,C2H2 zinc finger factors,Three-zinc finger Kruppel-related,COMPILED,P08047
...,...,...,...,...,...,...
70,MA0750.2,ZBTB7A,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,O95365
71,MA0748.2,YY2,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,O15391
72,MA1715.1,ZNF707,C2H2 zinc finger factors,More than 3 adjacent zinc fingers,ChIP-seq,Q96C28
73,MA0162.4,EGR1,C2H2 zinc finger factors,Three-zinc finger Kruppel-related,ChIP-seq,P18146


In [11]:
df_dp = pd.read_csv("6_DNA_protein.csv")
df_dp

,Matrix_ID,Uniprot_ID,DNA_Sequence,score/max_score,Label,protein_sequence
0,MA0002.1,Q01196,AATTGTGGTTA,0.995146,1,MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRS...
1,MA0002.1,Q01196,ATCTGTGGTTA,0.956311,1,MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRS...
2,MA0002.1,Q01196,TTCTGCGGTTA,0.922330,1,MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRS...
3,MA0002.1,Q01196,AATTGCGGTAA,0.917476,1,MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRS...
4,MA0002.1,Q01196,TATTGCGGTTT,0.932039,1,MRIPVDASTSRRFTPPSTALSPGKMSEALPLGAPDAGAALAGKLRS...
...,...,...,...,...,...,...
280805,MA1986.1,Q9BU19,GAACTTACATX,0.099733,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
280806,MA1986.1,Q9BU19,CAACAAACATX,0.077105,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
280807,MA1986.1,Q9BU19,ATTCGTTTATX,0.066380,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
280808,MA1986.1,Q9BU19,TTACATATAAX,0.087408,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...


In [22]:
training = df_dp[df_dp['Matrix_ID'].isin(train_df['ID'].tolist())]
testing = df_dp[df_dp['Matrix_ID'].isin(test_df['ID'].tolist())]
training = training.reset_index(drop=True)
testing = testing.reset_index(drop=True)

In [25]:
training

,Matrix_ID,Uniprot_ID,DNA_Sequence,score/max_score,Label,protein_sequence
0,MA0039.3,O43474,CCGCACCCTCC,0.882792,1,MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRQAGAPNNRWRE...
1,MA0039.3,O43474,CCACACCCTAA,0.978877,1,MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRQAGAPNNRWRE...
2,MA0039.3,O43474,CCACGCCCTCA,0.900052,1,MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRQAGAPNNRWRE...
3,MA0039.3,O43474,ACACACCCAAA,0.906878,1,MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRQAGAPNNRWRE...
4,MA0039.3,O43474,AGTCACCCTAA,0.763395,1,MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRQAGAPNNRWRE...
...,...,...,...,...,...,...
63261,MA1986.1,Q9BU19,GAACTTACATX,0.099733,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
63262,MA1986.1,Q9BU19,CAACAAACATX,0.077105,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
63263,MA1986.1,Q9BU19,ATTCGTTTATX,0.066380,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...
63264,MA1986.1,Q9BU19,TTACATATAAX,0.087408,0,MASSPAVDVSCRRREKRRQLDARRSKCRIRLGGHMEQWCLLKERLG...


In [14]:
with open('6_protein_embedding_sequence.json', 'r') as file:
    protein_embedding_sequence = json.load(file)
    
with open('6_one_hot_encoding_DNA.json', 'r') as file:
    one_hot_encoding_DNA = json.load(file)
    
with open('6_Word2Vec_encoding_DNA.json', 'r') as file:
    Word2Vec_encoding_DNA = json.load(file)

In [ ]:
peseq = protein_embedding_sequence[filtered_df['Uniprot_ID'][0]]
ohe_dna = one_hot_encoding_DNA[filtered_df['DNA_Sequence'][0]]
word2vec_dna = Word2Vec_encoding_DNA[filtered_df['DNA_Sequence'][0]]

training = {
    '0': peseq + word2vec_dna + [int(char) for char in ohe_dna]
}

training1_feature = pd.DataFrame.from_dict(training, orient='index')
for i in range(len(filtered_df)):
    print(i)
    peseq = protein_embedding_sequence[filtered_df['Uniprot_ID'][i]]
    ohe_dna = one_hot_encoding_DNA[filtered_df['DNA_Sequence'][i]]
    word2vec_dna = Word2Vec_encoding_DNA[filtered_df['DNA_Sequence'][i]]
    name = str(i)
    training1_feature.loc[name] = peseq + word2vec_dna + [int(char) for char in ohe_dna]

In [33]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix,accuracy_score

df = pd.read_csv("success_download.csv")
data = df[df['Class'] == 'C2H2 zinc finger factors']

df_dp = pd.read_csv("10.9_DNA_protein.csv")

train_df, test_df = train_test_split(data, test_size=9, random_state=10)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

training = df_dp[df_dp['Matrix_ID'].isin(train_df['ID'].tolist())]
testing = df_dp[df_dp['Matrix_ID'].isin(test_df['ID'].tolist())]
training = training.reset_index(drop=True)
testing = testing.reset_index(drop=True)

with open('protein_embedding_sequence.json', 'r') as file:
    protein_embedding_sequence = json.load(file)
    
with open('one_hot_encoding_DNA.json', 'r') as file:
    one_hot_encoding_DNA = json.load(file)
    
with open('Word2Vec_encoding_DNA.json', 'r') as file:
    Word2Vec_encoding_DNA = json.load(file)
   
peseq = protein_embedding_sequence[training['Uniprot_ID'][0]]
ohe_dna = one_hot_encoding_DNA[training['DNA_Sequence'][0]]
word2vec_dna = Word2Vec_encoding_DNA[training['DNA_Sequence'][0]]

training123 = {
    '0': peseq + word2vec_dna + [int(char) for char in ohe_dna]
}
'''
training1_feature = pd.DataFrame.from_dict(training123, orient='index')

training1_feature['Label'] = training['Label'].tolist()    
    
peseq2 = protein_embedding_sequence[testing['Uniprot_ID'][0]]
ohe_dna2 = one_hot_encoding_DNA[testing['DNA_Sequence'][0]]
word2vec_dna2 = Word2Vec_encoding_DNA[testing['DNA_Sequence'][0]]

testing123 = {
    '0': peseq2 + word2vec_dna2 + [int(char) for char in ohe_dna2]
}

testing1_feature = pd.DataFrame.from_dict(testing123, orient='index')
    
testing1_feature.index = pd.RangeIndex(start=63266, stop=70891)
testing1_feature['Label'] = testing['Label'].tolist()
'''

"\ntraining1_feature = pd.DataFrame.from_dict(training123, orient='index')\n\ntraining1_feature['Label'] = training['Label'].tolist()    \n    \npeseq2 = protein_embedding_sequence[testing['Uniprot_ID'][0]]\nohe_dna2 = one_hot_encoding_DNA[testing['DNA_Sequence'][0]]\nword2vec_dna2 = Word2Vec_encoding_DNA[testing['DNA_Sequence'][0]]\n\ntesting123 = {\n    '0': peseq2 + word2vec_dna2 + [int(char) for char in ohe_dna2]\n}\n\ntesting1_feature = pd.DataFrame.from_dict(testing123, orient='index')\n    \ntesting1_feature.index = pd.RangeIndex(start=63266, stop=70891)\ntesting1_feature['Label'] = testing['Label'].tolist()\n"